In [1]:
%cd /Users/mcortes/Documents/projects/cmm-bio/caidos/Markopy/mocbapy/mocbapy

/Users/mcortes/Documents/projects/cmm-bio/caidos/Markopy/mocbapy/mocbapy


In [2]:
import cobra.test
import mocbapy
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
import pandas as pd
from scipy.sparse import lil_matrix

# Ecosystem class

In [59]:
import cobra
import numpy as np

from cobra import Metabolite, Reaction, Model
from cobra.util.array import create_stoichiometric_matrix
from cobra.util.solver import linear_reaction_coefficients
from benpy import vlpProblem
from benpy import solve as bensolve
from scipy.sparse import lil_matrix
from scipy.spatial import Delaunay
from functools import reduce

class Ecosystem:

    def __init__(self, models, prefixes, community_name = "My community model", community_id= "community",
                 pool_bounds = (-1000,1000), keep_members=True, print_report = True):
        self.models = [m.copy() for m in models]
        self.size = len(models)
        self.name = community_name
        self.prefixes = prefixes
        self.conflicts = None
        self.cmodel = None
        
        # --- step 1 ---
        # Change rxn & met ids and compartments of member models to prefixes[i]:id, prefixes[i]:compartment
        #
        #  ex_mets: prefixes[i]:mex.id -> mex.id, where mex is a exchange metabolite in model i.    
        ex_mets = dict()
        for member_index in range(self.size):
            i_ex_mets = self._rename_member_model(member_index)
            ex_mets.update(i_ex_mets) 
        self.ex_mets = ex_mets
        
        # --- step 2 ---
        # 2.1 Store member models original objectives {rxn: coef}
        # 2.2 Store member models exchange reactions and their original bounds.
        self.objectives = dict()
        self.member_exchanges = list()
        for ix, m in enumerate(self.models):
            #2.1 objectives
            member_lr_coefs =  linear_reaction_coefficients(m)
            if len(member_lr_coefs) != 1:
                raise RuntimeError("More than one reaction in %s objective function. Not supported!!Set single objective reaction" 
                      % self.prefixes[ix])                
            member_objectives = { r.id:coeff for r,coeff in member_lr_coefs.items()} # should be single key,value dict   
            self.objectives.append(member_objectives)
            #2.2 exchange reactions
            exchanges = dict()
            for r in m.exchanges:
                exchanges[r.id] = r.bounds
            self.member_exchanges.append(exchanges)            
        
        # --- step 3 ---
        # Create new cobrapy Model for the community by merging member models
        cmodel = Model(id_or_model = community_id, name=community_name)
        for m in self.models:
            cmodel.merge(m)
            
        # --- step 4 ---
        # Store community model.
        self.cmodel = cmodel       
 
        # --- step 5 ---
        # Add pool to community model (including compartment, shared pool mets, pool exchanges
        #                              and update of member exchange reactions)
        # By default pool exchanges are set with bounds (-1000,1000). 
        # Exchange metabolites with same id but different formulas, charges or names among members are stored
        # in self.conflicts along with these values.
        
        self._add_pool(pool_bounds)
        
        # --- step 6 ----
        # Add compartment names. They get lost during merge (step 3)
        ccomps = self.cmodel.compartments
        for m in self.models:
            for comp in m.compartments:
                ccomps[comp] = m.compartments[comp]
        self.cmodel.compartments = ccomps
        
        self.cmodel.repair()
        # --- optional steps ----
        
        # Print summary report if print_report
        if print_report:
            self.print_build_report(pool_bounds)
        # Member models are kept unless keep_members is False:
        if not keep_members:
            self.clean_members()    
    
    def print_build_report(self, pool_bounds):
        """ Prints community construction report """
        models = self.models
        cmodel = self.cmodel
        prefixes = self.prefixes
        print("Created community model from %d member models:" % self.size)               
 
        print("General stats:")
        if models is not None:
            for i in range(self.size):
                print("model (%d):" % i) 
                print( "\t id = %s, name = %s , prefix= %s" % (models[i].id,models[i].name, prefixes[i]))
                rxns = len(models[i].reactions)
                comps = len(models[i].compartments)
                mex = 0
                for mid in self.ex_mets:
                    if mid.startswith(prefixes[i]):
                        mex +=1
                print( "\t\t reactions = %d" % rxns)
                print( "\t\t exchange metabolites = %d" % mex)
                print( "\t\t compartments = %d" % comps)
        print("community model:")
        print( "\t id = %s, name = %s " % (cmodel.id, cmodel.name))
        print( "\t\t reactions = %d" % len(cmodel.reactions))
        print( "\t\t exchange metabolites = %d" % len(self.pool_mets))
        print( "\t\t compartments = %d" % len(cmodel.compartments))        
        print("Exchange metabolite conflicts (formula, charge, name, id) = %d" % len(self.conflicts))   
        print("Community exchange reaction bounds: %s" % str(pool_bounds))
    
    def clean_members(self):
        """ Deletes member individual models"""
        members = self.models
        self.models = None
        del members
        
    def _rename_member_model(self, member_index):
        """ Updates a member model by renaming its metabolite and reaction ids and also 
            its compartment names and ids. 
            The corresponding community member prefixes are used:
                original_rxn_id --> member_prefix:original_rxn_id
                original_met_id --> member_prefix:original_met_id
                original_compartment_id --> member_prefix:original_compartment_id
                original_compartment_name --> member_prefix original_compartment_name

            member_index: index of member model in self.models list. 
                    Draft community model corresponding to a merge of individual models 
                    (with translated ids).
            ex_mets: dictionary 
                     keys: translated exchange metabolite ids (e.g. model1:glyc_e, model2:glyc_e).
                     values: original exchange metabolite ids (e.g. glyc_e, glyc_e)

            Returns:       
            exchange_mets : dict
                                keys: translated ids of exchange metabolites in member model
                                values: original ids of exchange metabolites in member model            
        """        
        
        model = self.models[member_index]
        model_prefix = self.prefixes[member_index]
        comp_dict = dict()
        exchange_mets = dict()
        
        # Building exchange metabolites dictionary of translated to original ids
        for rex in model.exchanges:
            for mex in rex.metabolites:
                new_mex_id = "%s:%s" % (model_prefix,mex.id)            
                exchange_mets[new_mex_id]= mex.id

        # Updating metabolite ids        
        for m in model.metabolites:
            mid = "%s:%s" % (model_prefix,m.id)
            m.id = mid
            mcomp = "%s:%s" % (model_prefix,m.compartment)
            if mcomp not in comp_dict:
                comp_dict[mcomp]= "%s %s"   % (model_prefix, model.compartments[m.compartment])
            m.compartment = mcomp
        model.repair()    
        model.compartments = comp_dict

        #Updating reaction ids
        for rxn in model.reactions:
            rid = "%s:%s" % (model_prefix,rxn.id)
            rxn.id = rid
        model.repair()

        return exchange_mets

    def _build_pool_metabolites(self):
        """ Builds community pool metabolites by getting metabolites names, formulas and charges from 
            individual models. Conflicts where more than one value for either metabolite 
            formula, charge or name are found are stored and returned.
            A list of Metabolite objects for pool metabolites is generated.

            self.cmodel: cobra.Model 
                    Draft community model corresponding to a merge of individual models 
                    (with translated ids).
            self.ex_mets: dictionary 
                     keys: translated exchange metabolite ids (e.g. model1:glyc_e, model2:glyc_e).
                     values: original exchange metabolite ids (e.g. glyc_e, glyc_e)

            Returns:   

            pool_mets: list
                       Elements of pool_mets are cobra.Metabolite object for each pool metabolite.    
            balance_conflicts : dict
                                keys: pool metabolites ids of pool metabolites with different 
                                formula,charge or name values.
                                values: dict with formula, charge and name lists.            
        """

        cmodel =self.cmodel
        ex_mets = self.ex_mets
        
        pool_mets = []
        balance_conflicts = dict()

        # Pool exchange metabolites original ids in community member models:
        ex_original_ids = set(ex_mets.values())

        # dictionary to store formulas, charges and names for each exchange metabolite:     
        ex_metadata = {mid: {"formulas":set(), "charges":set(),"names":set()} for mid in ex_original_ids} 

        for ex_new, ex_original in ex_mets.items():
            
            m = cmodel.metabolites.get_by_id(ex_new)
                        
            ex_metadata[ex_original]["formulas"].add(m.formula)
            ex_metadata[ex_original]["charges"].add(m.charge)
            ex_metadata[ex_original]["names"].add(m.name) 
            

        # Checking if each exchange metabolites has one or more value por formula, charge, or name: 
        for ex_original,ex_data in ex_metadata.items():    
            formulas =  ex_data['formulas']
            charges  =  ex_data['charges']
            names    =  ex_data['names']

            if len(formulas)!=1 or len(charges) != 1 or len(names) != 1:
                balance_conflicts[ex_original] = ex_data 
                print("Conflicts for exchange metabolite %s:" % ex_original)
                print("Formulas:")
                print(formulas)
                print("Charges:")
                print(charges)
                print("Names:")
                print(names)
                print("----------")


            # Setting id, formula,charge and name for new pool metabolite
            eid = "%s:pool" % ex_original
            eformula = "X"  if len(formulas)== 0 else formulas.pop()
            echarge = "0"  if len(charges)== 0 else charges.pop()
            ename = ex_original if len(names)== 0 else names.pop()

            # pool metabolite Metabolite object
            mpool = Metabolite(eid, formula = eformula, charge = echarge, name = ename, compartment = 'pool')
            pool_mets.append(mpool)            

        return pool_mets, balance_conflicts   

    def _build_pool_reactions(self, pool_bounds = (-1000,1000)):
        """ Builds community exchange reactions to ad to the community model. 
            Default reaction bounds are set as (-1000,1000) for each community exchange reaction."""
        
        
        cmodel = self.cmodel
        pool_mets = self.pool_mets
        
        pool_rxns = list()
        # One exchange reaction for each pool metabolite. 
        for mid in pool_mets:
            m = self.cmodel.metabolites.get_by_id(mid)
            ex_rxn = Reaction(
                id   = "EX_%s" % m.id,
                name = "Pool %s exchange" % m.name,
                subsystem = "Exchange",
                lower_bound = pool_bounds[0],
                upper_bound = pool_bounds[1],
                objective_coefficient = 0.0
            )
            ex_rxn.add_metabolites({m: -1.0})        
            pool_rxns.append(ex_rxn)
        return pool_rxns

    def _add_pool(self,pool_bounds = (-1000,1000)):
        """ 
        Adds pool to community model (including compartment, shared pool mets, pool exchanges
                                      and update of member exchange reactions)
         By default pool exchanges are set with bounds (-1000,1000)
         Exchange metabolites with same id but different formulas, charges or names among members are stored
         in conflicts along with these values.        
        """
        cmodel =self.cmodel
        ex_mets = self.ex_mets
        
        compartments = cmodel.compartments
        compartments['pool']="Community pool"


        #list of pool metabolites to add and conflict report dict
        pool_mets, conflicts = self._build_pool_metabolites()
        self.conflicts = conflicts

        #adding pool metabolites to community model    
        cmodel.add_metabolites(pool_mets) 
        self.pool_mets= [m.id for m in pool_mets]
        #cmodel.repair()

        #list of exchange reactions to add for each pool metabolite
        pool_exchange_reactions = self._build_pool_reactions(pool_bounds)    

        #updating original exchange reactions from member models:
        # from met_e:model1 <--> to met_e:model1 <--> met_e:pool
        member_exchanges = self.member_exchanges
        for ex_dict in member_exchanges:
            for ex_id in ex_dict:
                ex = cmodel.reactions.get_by_id(ex_id)
                for mex in ex.metabolites:
                    coeff =  ex.metabolites[mex]
                    mpool_id = "%s:pool" % ex_mets[mex.id]
                    mpool = cmodel.metabolites.get_by_id(mpool_id)
                    ex.add_metabolites({mpool: -coeff})
               
        #adding exchange reactions to community model
        cmodel.add_reactions(pool_exchange_reactions)

        #updating compartment names:
        cmodel.compartments = compartments
           
        #storing pool exchange reactions:
        self.pool_reactions = [r.id for r in pool_exchange_reactions]
    
    def set_medium(self, exchange_bounds):
        for rid in exchange_bounds:
            pool_exchange_reaction = self.cmodel.reactions.get_by_id(rid)
            pool_exchange_reaction.bounds = exchange_bounds[rid]
            
    def _to_vlp(self,**kwargs):        
        """Returns a vlp problem from EcosystemModel"""
        # We are using bensolve-2.0.1:
        # B is coefficient matrix
        # P is objective Matrix
        # a is lower bounds for B
        # b is upper bounds for B
        # l is lower bounds of variables
        # s is upper bounds of variables
        # opt_dir is direction: 1 min, -1 max
        # Y,Z and c are part of cone definition. If empty => MOLP
        
        cmodel = self.cmodel
        Ssigma = create_stoichiometric_matrix(cmodel, array_type="lil")
        
        vlp = vlpProblem(**kwargs)
        m, n = Ssigma.shape # mets, reactions
        q = self.size # number of members 
        vlp.B = Ssigma
        vlp.a = np.zeros((1, m))[0]
        vlp.b = np.zeros((1, m))[0]
        vlp.l = [r.lower_bound for r in cmodel.reactions] 
        vlp.s = [r.upper_bound for r in cmodel.reactions] 
        
        
        vlp.P = lil_matrix((q, n))
        vlp.opt_dir = -1
        
        for i, member_objectives in enumerate(self.objectives):
            for rid, coeff in member_objectives.items():
                rindex = cmodel.reactions.index(rid)
                vlp.P[i,rindex] = coeff 
                
        vlp.Y = None
        vlp.Z = None
        vlp.c = None
        return vlp  
    
    def mo_fba(self,**kwards):
        vlp_eco = self._to_vlp(**kwards)
        self.mo_fba_sol = bensolve(vlp_eco)
    
    def get_polytope_vertex(self):
  
        """
        polytope: pareto front + axes segments + extra segments perpendicular to axes dimensions where 
        pareto solutions don't reach 0 values. 
        (assumption: objective functions can only take positive values)
        """
        
        #1. Front vertex:
        vv = self.mo_fba_sol.Primal.vertex_value[np.array(self.mo_fba_sol.Primal.vertex_type)==1]
        #2. origin
        ov = np.zeros((1,self.size))
        
         
        #3. Extra points that close polytope (only if points (0,0,0,...,xi_max,0,...0) are not pareto front 
        # points but they are feasible) 
        
        #MP: si hay que incluir estos puntos significa que hay miembros que son givers: i.e. pueden crecer
        # a su máxima tasa y aguantar que otros miembros crezcan igual
        # si un punto (0,0,0,...,xi_max,0,...0) no es factible entonces el miembro i necesita que otros crezcan 
        # para poder crecer (needy).
        
        #3.1 Check if points  (0,0,0,...,xi_max,0,...0) are part of the pareto front
        n = self.size - 1
        all_zeros_but_one = np.argwhere(np.sum(vv == 0,axis=1)==n) # index of (0,0,...,xi_max,0,0...0) points
        all_zeros_but_one = all_zeros_but_one.flatten()

        # indexes i of non-zero member in (0,0,...,xi_max,0,0...0) pareto points, 
        # i.e. members that are not givers nor needy. 
        non_zero_dims =  np.argmax(vv[all_zeros_but_one,:], axis = 1) 
        
        # givers and/or needy members:
        givers_or_needy_indexes = np.setdiff1d(np.array(range(self.size)), non_zero_dims) 
        gn_total= len(givers_or_needy_indexes)    
    
        #3.2 Check if non-pareto points (0,0,0,...,xi_max,0,...0) are feasible 
        if gn_total >0:
            # max values for giver_or_needy members:
            max_vals = np.max(vv, axis=0)
            cpoints = np.diag(max_vals)
            to_check = cpoints[givers_or_needy_indexes,:]
            
            are_feasible = self.check_feasible(to_check)
            
            ev = to_check[are_feasible,:] 
            polytope_vertex = np.concatenate((vv,pv,ev), axis=0)
        else: 
            polytope_vertex = np.concatenate((vv,pv), axis=0)
                     
        return polytope_vertex 
    
    def check_feasible(self, point_array):
        r = [self.check_feasible_point(p) for p in point_array] 
        return r
               
    def check_feasible_point(self,point):
        cmodel = self.cmodel
        
        original_bounds = dict()
        original_community_objective =  cmodel.objective
        
        for ix, member_objectives in enumerate(self.objectives):    
            if len(member_objectives) != 1:
                raise RuntimeError("Warning: More than one reaction in %s objective function. Not supported!!" 
                      % self.prefixes[ix])        
            #new bounds for member ix objective function reaction:     
            new_bounds = (point[ix],point[ix])    
                         
            #change bounds for each objective reaction and store original bounds
            for rid in member_objectives.keys(): # member_objectives should be single key dictionary
                rxn = cmodel.reactions.get_by_id(rid)
                old_bounds = self.change_reaction_bounds(rid, new_bounds)
                original_bounds[rid] = old_bounds 
            #set one of objective reactions as community objective, e.g. the non-zero one in point
                if point[ix]>0:
                    cmodel.objective = rid
            
        #check point feasibility
        error_value = -1000
        ob = cmodel.slim_optimize(error_value = error_value)
        if ob == error_value:
            is_feasible = False
        else:
            is_feasible = True
        # return everything as it was originally    
        cmodel.objective = original_community_objective   
        for rid, old_bounds in original_bounds.items():
            nbounds = self.change_reaction_bounds(rid, old_bounds )
        
        return is_feasible
            
    def change_reaction_bounds(self, rid, new_bounds):
        cmodel = self.cmodel
        rxn = cmodel.reactions.get_by_id(rid)
        old_bounds = rxn.bounds
        rxn.bounds = new_bounds
        return old_bounds
            
    def plot_polytope(self):

       # from matplotlib.tri import triangulation
       # from scipy.spatial import ConvexHull
       # 
       # # compute the convex hull of the points
       # cvx = ConvexHull(X)
       # 
       # x, y, z = X.T
       # 
       # # cvx.simplices contains an (nfacets, 3) array specifying the indices of
       # # the vertices for each simplical facet
       # tri = Triangulation(x, y, triangles=cvx.simplices)
       # 
       # fig = plt.figure()
       # ax = fig.gca(projection='3d')
       # ax.hold(True)
       # ax.plot_trisurf(tri, z)
       # ax.plot_wireframe(x, y, z, color='r')
       # ax.scatter(x, y, z, color='r')
       # plt.draw()    
            return
    
    def build_grid(self,step = 0.1):
        
        #polytope vertex
        polytope_vertex = self.get_polytope_vertex()
        
        #polytope from vertex
        hull = Delaunay(polytope_vertex)
        
        # "rectangular" grid
        maxs = np.max(polytope_vertex, axis=1)
        mins = np.min(polytope_vertex, axis=1) #should be vector of zeros
        size = self.size
        
        
        slices = [slice(mins[i],maxs[i],step) for i in range(size)]
        rgrid = np.mgrid[slices]
        rgrid2columns = [rgrid[i,:].ravel() for i in range(size)]
        # array of grid points (x,y,z,...)
        positions = np.column_stack(rgrid2columns)
        #polytope intersection    
        inside = hull.find_simplex(positions)>=0
        
        # storing grid points inside polytope
        self.points = positions[inside] 
        self.step = step
        self.limits = (mins,maxs)
    
    def get_2D_slice(self, prefixes, fixed_values):
        if len(prefixes) != self.size-2:
            raise RuntimeError("Only two members with non-fixed values required!")
        
        to_fix = range(len(prefixes))
        closest_values = [self._get_closest_grid_value(prefixes[i],fixed_values[i])for i in to_fix]
        fixed_indexes = [self.prefixes.index(x) for x in prefixes]
        
        
        grid_points = self.points
        #indices de puntos en grilla donde el valor una dimension es igual a su closest_values
        filtered_indexes_list =  [np.where(grid_points[:,fixed_member_indexes[i]] == closest_values[i]) for i in to_fix]
        
        #interseccion de indices, i.e., indices slice
        filtered_indexes =  reduce(np.intersect1d, filtered_indexes_list)
        
        #slice 2D de la grilla
        filtered_points = grid_points[filtered_indexes,:] 
        slice_points = filtered_points[:, ]

        
        
        
    def _get_closest_grid_value(self,prefix,fixed_value):
        member_index = self.prefixes.index(prefix)
        member_min = self.limits[0][member_index]
        member_max = self.limits[1][member_index]
        
        if fixed_value < member_min or fixed_value > member_max:
            raise RuntimeError("Value %d for %s out of range" % (fixed_value, prefix))
        
        
        shifted_value = fixed_value - member_min
        n_steps = shifted_value//self.step          
        
        p1 = n_steps * self.step
        p2 = (n_steps + 1) * self.step  
        
        if (shifted_value - p1) < (p2 - shifted_value):
            closest_value = member_min + p1
        else:
            closest_value = member_min + p2        
        
        return closest_value
    

# Tests

In [40]:
test_models = list()
test_prefixes = list()

n_test = 2
for i in range(n_test):
    model = cobra.test.create_test_model("ecoli")
    if i == 1:
        r = model.reactions.get_by_id('ACGK')
        r.objective_coefficient = 1
    mid = 'coli_' + str(i+1)
    test_models.append(model)
    test_prefixes.append(mid)

In [41]:
test_eco = Ecosystem(test_models, test_prefixes)

RuntimeError: More than one reaction in coli_2 objective function. Not supported!!
 Set single objective reaction

In [27]:
test_eco.objectives

[{'coli_1:Ec_biomass_iJO1366_core_53p95M': 1.0},
 {'coli_2:Ec_biomass_iJO1366_core_53p95M': 1.0, 'coli_2:ACGK': 1.0}]

In [28]:
test_eco.set_medium({'EX_glc__D_e:pool':(-20, 1000)})

In [29]:
bensolve_opts = mocbapy.bensolve_default_options
bensolve_opts['message_level'] = 0
sol_mofba = test_eco.mo_fba(options=bensolve_opts)
print(sol_mofba)

/var/folders/hv/r4rp10lx5nnc3q1x5ht0cl2w0000gn/T/tmp4258y05f
None


/Users/mcortes/anaconda/envs/markopy/lib/python3.6/site-packages/ipykernel_launcher.py:367 UserWarning: 
Pre image was not saved, preimage value set to None. Include 'solution':True in problem options dictionary


In [30]:
vv = sol_mofba.Primal.vertex_value[np.array(sol_mofba.Primal.vertex_type)==1]

AttributeError: 'NoneType' object has no attribute 'Primal'

In [62]:
reduce(np.intersect1d, [[1, 3, 4, 3], [3, 1, 4, 1], [6, 3, 4, 2]])

array([3, 4])

In [37]:
test_eco.vlp.Primal.vertex_value

array([[  0.98237181, 101.74      ],
       [ -1.        ,   0.        ],
       [  0.        ,  -1.        ]])

In [38]:
test_eco.objectives

[{'coli_1:Ec_biomass_iJO1366_core_53p95M': 1.0},
 {'coli_2:Ec_biomass_iJO1366_core_53p95M': 1.0, 'coli_2:ACGK': 1.0}]

In [42]:
maxs = np.array([3.33,4.4,5.5])
mins = np.array([0,0,0])
size = 3
step=0.5

slices = [slice(mins[i],maxs[i],step) for i in range(size)]
rgrid = np.mgrid[slices]
rgrid2columns = [rgrid[i,:].ravel() for i in range(size)]
positions = np.column_stack(rgrid2columns)



In [43]:
positions.shape

(693, 3)

In [44]:
positions

array([[0. , 0. , 0. ],
       [0. , 0. , 0.5],
       [0. , 0. , 1. ],
       ...,
       [3. , 4. , 4. ],
       [3. , 4. , 4.5],
       [3. , 4. , 5. ]])

In [55]:
x = np.where((positions[:,0]==2) & (positions[:,1]==4))

In [56]:
positions[x,:]

array([[[2. , 4. , 0. ],
        [2. , 4. , 0.5],
        [2. , 4. , 1. ],
        [2. , 4. , 1.5],
        [2. , 4. , 2. ],
        [2. , 4. , 2.5],
        [2. , 4. , 3. ],
        [2. , 4. , 3.5],
        [2. , 4. , 4. ],
        [2. , 4. , 4.5],
        [2. , 4. , 5. ]]])

In [64]:
s = ['a','b','c','d']
s[[-3,-2]]

TypeError: list indices must be integers or slices, not list